Transition model from development to staging

In [1]:
exp_name = "fashion-mnist"

In [2]:
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import os
from mlflow.entities import ViewType
from dotenv import load_dotenv
load_dotenv()
os.getenv("MLFLOW_TRACKING_URI")


'http://localhost:5000'

In [3]:

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

mlflow.set_experiment(experiment_name=exp_name)

<Experiment: artifact_location='file:///C:/Users/SGU/Projects/MLFlow/fashion-mnist/artifacts/2', creation_time=1693562843111, experiment_id='2', last_update_time=1693562843111, lifecycle_stage='active', name='fashion-mnist', tags={}>

In [4]:
exp = mlflow.search_experiments(filter_string=f"name = '{exp_name}'")

runs = MlflowClient().search_runs(
    experiment_ids=[exp[0].experiment_id],
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.f1 DESC"]
)
best_run = runs[0].to_dictionary()
best_run_id = best_run['info']['run_id']
print(best_run)
best_run_id

{'info': {'artifact_uri': 'file:///C:/Users/SGU/Projects/MLFlow/fashion-mnist/artifacts/2/1282763d654c47509e9802b02c35a89f/artifacts', 'end_time': 1693563019120, 'experiment_id': '2', 'lifecycle_stage': 'active', 'run_id': '1282763d654c47509e9802b02c35a89f', 'run_name': 'CNN', 'run_uuid': '1282763d654c47509e9802b02c35a89f', 'start_time': 1693563006538, 'status': 'FINISHED', 'user_id': 'SGU'}, 'data': {'metrics': {'precision': 0.8757323726297757, 'recall': 0.87, 'f1': 0.8681048688024267}, 'params': {'model_name': 'CNN', 'lr': '0.001', 'batch_size': '100'}, 'tags': {'mlflow.user': 'SGU', 'mlflow.source.name': 'c:\\Users\\SGU\\Miniconda3\\envs\\part_segmentation_mmdetection\\lib\\site-packages\\ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.runName': 'CNN', 'mlflow.log-model.history': '[{"run_id": "1282763d654c47509e9802b02c35a89f", "artifact_path": "model", "utc_time_created": "2023-09-01 10:10:12.874883", "flavors": {"pytorch": {"model_data": "data", "pytorch_version": "

'1282763d654c47509e9802b02c35a89f'

In [5]:
client = MlflowClient()
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1693504416537, description='', last_updated_timestamp=1693563343131, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1693504416569, current_stage='Staging', description='', last_updated_timestamp=1693504492603, name='CNN', run_id='81770435ab5c409b9e6faa3a3a13c83f', run_link='', source='file:///C:/Users/SGU/Projects/MLFlow/fashion-mnist/artifacts/1/81770435ab5c409b9e6faa3a3a13c83f/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>,
  <ModelVersion: aliases=[], creation_timestamp=1693563343131, current_stage='None', description='', last_updated_timestamp=1693563343131, name='CNN', run_id='e99a12f64e584c9993e5ab9cfe510734', run_link='', source='file:///C:/Users/SGU/Projects/MLFlow/fashion-mnist/artifacts/2/e99a12f64e584c9993e5ab9cfe510734/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>], name='CNN', tags={}>,
 <RegisteredModel: aliases={}, creation_ti

In [6]:
client = MlflowClient()
rm = client.search_model_versions(filter_string=f"run_id='{best_run_id}'", max_results=1)
model_version = dict(rm[0])
pprint(model_version, indent=4)
name = model_version["name"]
version = model_version["version"]
rm

{   'aliases': [],
    'creation_timestamp': 1693563019093,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1693563019093,
    'name': 'CNN',
    'run_id': '1282763d654c47509e9802b02c35a89f',
    'run_link': '',
    'source': 'file:///C:/Users/SGU/Projects/MLFlow/fashion-mnist/artifacts/2/1282763d654c47509e9802b02c35a89f/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}


[<ModelVersion: aliases=[], creation_timestamp=1693563019093, current_stage='None', description='', last_updated_timestamp=1693563019093, name='CNN', run_id='1282763d654c47509e9802b02c35a89f', run_link='', source='file:///C:/Users/SGU/Projects/MLFlow/fashion-mnist/artifacts/2/1282763d654c47509e9802b02c35a89f/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>]

In [7]:
client.transition_model_version_stage(name=name,version=version,stage="Staging")

<ModelVersion: aliases=[], creation_timestamp=1693563019093, current_stage='Staging', description='', last_updated_timestamp=1693563578823, name='CNN', run_id='1282763d654c47509e9802b02c35a89f', run_link='', source='file:///C:/Users/SGU/Projects/MLFlow/fashion-mnist/artifacts/2/1282763d654c47509e9802b02c35a89f/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

Serving model from model registry

In [8]:
import pandas as pd
test_csv = pd.read_csv(r"data/fashion-mnist_test.csv")
X_test = test_csv.drop('label', axis=1)
y_test = test_csv['label']


In [9]:
import torch
import numpy as np
X_test_np = X_test.to_numpy()
input = np.asarray(X_test_np).reshape(-1, 1, 28, 28).astype('float32')
input.shape

c:\Users\SGU\Miniconda3\envs\part_segmentation_mmdetection\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(10000, 1, 28, 28)

In [13]:
import mlflow.pyfunc

stage = 'Staging'

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{name}/{stage}"
)
y_pred = model.predict(input)
predicted = np.argmax(y_pred, 1)
predicted

c:\Users\SGU\Miniconda3\envs\part_segmentation_mmdetection\lib\site-packages\torch\nn\functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


array([0, 1, 2, ..., 8, 8, 1], dtype=int64)

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predicted)
cm

array([[888,   2,   5,  26,   4,   2,  60,   0,  13,   0],
       [  2, 983,   0,   6,   1,   2,   6,   0,   0,   0],
       [ 32,   2, 752,   7, 100,   0, 100,   0,   7,   0],
       [ 44,  24,   1, 848,  43,   0,  40,   0,   0,   0],
       [  1,   4,  46,  17, 864,   0,  66,   0,   2,   0],
       [  0,   1,   0,   0,   0, 952,   0,  10,   2,  35],
       [229,   6,  44,  14, 119,   0, 576,   0,  12,   0],
       [  0,   0,   0,   0,   0,  74,   0, 719,   0, 207],
       [  3,   3,   4,   4,   4,   5,   6,   1, 964,   6],
       [  0,   0,   0,   0,   0,   1,   0,   2,   0, 997]], dtype=int64)

In [12]:
# #http://127.0.0.1:1234

# import requests

# inference_request = {
#         "dataframe_records": input.tolist()
# }
# endpoint = "http://127.0.0.1:1234/invocations"
# response = requests.post(endpoint, json=inference_request)
# print(response)